In [46]:
import tensorflow as tf

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !pip install tensorflow_text
# !pip install transformers
# !pip install emoji


In [76]:
import numpy as np
import pandas as pd
import plotly.express as px
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.layers import LSTM
#from google.colab import drive
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from transformers import TFBertForSequenceClassification
from tensorflow import keras
import os
import shutil
#import tensorflow as tf
#import tensorflow_hub as hub
#import tensorflow_text as text
#from official.nlp import optimization  # to create AdamW optimizer
from sklearn.model_selection import train_test_split
from transformers import TFAutoModel, AutoTokenizer
import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')

In [5]:
#drive.mount('/content/drive')

In [6]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='latin-1',header=None)

In [7]:
df[0] = df[0].apply(lambda x : 1 if x > 0 else 0)

In [8]:
df = shuffle(df)

In [9]:
test = df[500:1000]#df[1280000:1600000]
df = df[0:500]#df[:1280000]

In [10]:
test

,0,1,2,3,4,5
758508,0,2295762415,Tue Jun 23 08:20:37 PDT 2009,NO_QUERY,pamela__ann,make me feel good when I hurt so bad
452417,0,2070019408,Sun Jun 07 16:57:26 PDT 2009,NO_QUERY,MyGodreigns,"relaxing after a long, busy weekend and alread..."
339814,0,2014720063,Wed Jun 03 02:27:47 PDT 2009,NO_QUERY,diehardtryhard,"@gelurr i miss you, too, angel! don't worry a..."
79248,0,1751747146,Sat May 09 20:10:42 PDT 2009,NO_QUERY,creepyblindy,"@steveofmaine very sad, i love burgers"
926719,1,1755742257,Sun May 10 09:54:43 PDT 2009,NO_QUERY,jtcal,I knew we'd get May Gray weather don't mind it...
...,...,...,...,...,...,...
702548,0,2255428281,Sat Jun 20 11:14:55 PDT 2009,NO_QUERY,sunshinelayouts,all this #iranelection sstuff is so depressing
1067205,1,1965449257,Fri May 29 15:56:32 PDT 2009,NO_QUERY,carolinachic919,@jeanandyj It'll be a double 4 me cause Charle...
1494063,1,2069622957,Sun Jun 07 16:13:03 PDT 2009,NO_QUERY,m_mild,loves flying the SWISS way
1555329,1,2185100821,Mon Jun 15 16:53:14 PDT 2009,NO_QUERY,tish_nae,@_Dominque Ohhhh! Okay. . . Just let me know a...


In [11]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

In [12]:
arr = df[0].values  # take sentiment column in df as array
labels = np.zeros((arr.size, arr.max()+1))  # initialize empty (all zero) label array
labels[np.arange(arr.size), arr] = 1  # add ones in indices where we have a value

In [13]:
SEQ_LEN = 65
def tokenize(sentence):
    tokens = tokenizer.encode_plus(sentence, max_length=SEQ_LEN,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_attention_mask=True,
                                   return_token_type_ids=False, return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

In [14]:
Xids = np.zeros((len(df), SEQ_LEN))
Xmask = np.zeros((len(df), SEQ_LEN))

In [15]:
for i, sentence in enumerate(df[5]):
    Xids[i, :], Xmask[i, :] = tokenize(sentence)

In [16]:
with open('xids.npy','wb') as f:
  np.save(f, Xids)

with open('xmask.npy', 'wb') as f:
  np.save(f, Xmask)

with open('labels.npy', 'wb') as f:
  np.save(f, labels)

del df, Xids, Xmask, labels

In [17]:
with open('xids.npy','rb') as fp:
  Xids = np.load(fp)

with open('xmask.npy', 'rb') as fp:
  Xmask = np.load(fp)

with open('labels.npy', 'rb') as fp:
  labels = np.load(fp)



In [18]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))


In [19]:
# restructure dataset format for BERT
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels
  
dataset = dataset.map(map_func)  # apply the mapping function

In [20]:
# shuffle and batch the dataset
dataset = dataset.shuffle(10000).batch(32)

DS_LEN = len(list(dataset))  # get dataset length

SPLIT = 0.9  # we will create a 90-10 split

# create training-validation sets
train = dataset.take(round(DS_LEN*SPLIT))
val = dataset.skip(round(DS_LEN*SPLIT))

In [21]:
# build the model
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

#input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
#mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

In [22]:
bertweet = TFAutoModel.from_pretrained("vinai/bertweet-base")

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [23]:
embeddings = berttweet.roberta(input_ids, attention_mask=mask)[0]  # we only keep tensor 0 (last_hidden_state)

X = tf.keras.layers.GlobalMaxPool1D()(embeddings)  # reduce tensor dimensionality
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
X = tf.keras.layers.Dense(32, activation='relu')(X)
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(X)  # adjust based on number of sentiment classes



In [24]:
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

In [25]:
model.layers[2].trainable = False

In [26]:
optimizer = tf.keras.optimizers.Adam(0.01)
loss = tf.keras.losses.BinaryCrossentropy()
acc = tf.keras.metrics.BinaryAccuracy('accuracy')

In [27]:
model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [28]:
history = model.fit(train,
                    validation_data = val,
                    epochs=1)

14/14 [==============================] - 38s 2s/step - loss: 0.8805 - accuracy: 0.5580 - val_loss: 0.7114 - val_accuracy: 0.6731


In [29]:
#tf.keras.models.save_model(model, 'drive/MyDrive/twitter-sentiment-model-2', overwrite=True, include_optimizer=True, save_format="tf")


In [66]:
loaded_model = tf.keras.models.load_model('twitter-sentiment-model-2')

In [68]:
loaded_model

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 65)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 65)]         0                                            
__________________________________________________________________________________________________
roberta (TFRobertaMainLayer)    TFBaseModelOutputWit 134899968   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
global_max_pooling1d_4 (GlobalM (None, 768)          0           roberta[0][0]              

In [69]:
single_test_ids = np.zeros((len(test), SEQ_LEN))
single_test_mask = np.zeros((len(test), SEQ_LEN))

In [70]:
for i, sentence in enumerate(test[5]):
    print(sentence)
    single_test_ids[i, :], single_test_mask[i, :] = tokenize(sentence)




make me feel good when I hurt so bad 
relaxing after a long, busy weekend and already missing my princess 
@gelurr i miss you, too, angel!  don't worry about it noh, it's okay.
@steveofmaine  very sad, i love burgers
I knew we'd get May Gray weather don't mind it at all. Good luck Dodgers, Happy Mother's Day Mom and to everyone who is a Mom. 
@CBD126 Your witticisms are getting old 
so idk lol. but i know the pic of me &amp; chelsea staub is in it lol. Thanks @BrittneyTee for telling me  i didn't even know lol
@YoungQ Get some sleep, man!    LOL.....Blockdreamers IN EFFECT!  
I already miss my kitty  
@Logic187 sounds like my life! 
Out with the loves of my life. Enjoying The Weather!! 
just been dumped by hott guy 
I'm in a pretty bad mood and I have no damn idea why! Grr! 
rb@karensemb with thanks  â« http://blip.fm/~7fwyt
@bmthofficial why dnt u guys come to jkt?why you guys cancelled it? 
Here's a demo of the Community2.0 #WordPress theme http://mohdrafie.co.uk/labs/ 
@selenagomez

In [71]:
predictions = loaded_model.predict([single_test_ids, single_test_mask])

In [72]:
pred_label = []
for pred in predictions:
    pred_label.append(np.argmax(pred))


In [73]:
accuracy_score(pred_label, test[0])

0.48

In [75]:
predictions

array([[0.08576137, 0.9142387 ],
       [0.06131179, 0.9386882 ],
       [0.05247486, 0.94752514],
       [0.13554336, 0.86445665],
       [0.10738838, 0.8926116 ],
       [0.09959347, 0.90040654],
       [0.06928442, 0.93071556],
       [0.09197912, 0.90802085],
       [0.1629416 , 0.8370584 ],
       [0.13666515, 0.8633349 ],
       [0.07084195, 0.92915803],
       [0.1297037 , 0.87029636],
       [0.14788185, 0.85211813],
       [0.09234226, 0.90765774],
       [0.08824974, 0.9117503 ],
       [0.10439239, 0.8956076 ],
       [0.08194255, 0.9180575 ],
       [0.08610679, 0.91389316],
       [0.11854487, 0.88145506],
       [0.07939241, 0.9206076 ],
       [0.19229183, 0.80770814],
       [0.09120353, 0.9087965 ],
       [0.06190569, 0.9380943 ],
       [0.14884375, 0.85115623],
       [0.06791432, 0.9320857 ],
       [0.11854839, 0.88145167],
       [0.15617512, 0.84382486],
       [0.09649368, 0.90350634],
       [0.11201282, 0.8879872 ],
       [0.05072775, 0.9492723 ],
       [0.